# Train

In [4]:
import os
import random
import numpy as np
import torch
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from torch import nn
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
import config
import utils
from model import DependencyBLSTM

args = config.args
output_dir = args.project_dir + 'Models/' + args.sim_name + '/'

utils.creat_word_embedding()

def run():
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    save_path = args.project_dir + 'Models/' + args.sim_name + '/model.pt'

    with open(output_dir + 'config', 'w') as config_file:
        argss = (str(args).split('(')[1].split(')')[0].split(','))
        for a in argss:
            config_file.write("{}\n".format(a))
    if os.path.exists(save_path):
        model = torch.load(save_path)
        model_loaded = True
        print('Great!!! Pre-Trained Model Loaded !!!')
    else:
        model_loaded = False
        print('No pre-trained model ')
        model = DependencyBLSTM(num_words=utils.get_num_words(), max_sen_length=97, max_doc_sent_length=326)

    if not os.path.exists(output_dir + 'train_performance_log.csv'):
        train_performance_log = open(output_dir + 'train_performance_log.csv', 'w')
        train_performance_log.write('Step,Loss\n')
    else:
        train_performance_log = open(output_dir + 'train_performance_log.csv', 'a')

    if not os.path.exists(output_dir + 'eval_performance_log.txt'):
        eval_performance_log = open(output_dir + 'eval_performance_log.txt', 'w')
    else:
        eval_performance_log = open(output_dir + 'eval_performance_log.txt', 'a')

    if args.gpu > -1:
        model.cuda(device=int(args.gpu))

    if not model_loaded:
        if args.fill_embedding:
            embed = utils.get_word_embeddings(source='google')
            if args.gpu > -1:
                model.word_embedding.weight.data.set_(torch.FloatTensor((embed)).cuda(int(args.gpu)))
            else:
                model.word_embedding.weight.data.set_(torch.FloatTensor((embed)))
        else:
            if args.gpu > -1:
                model.word_embedding.weight.data.set_(
                    torch.FloatTensor((np.zeros((utils.get_num_words() + 1, args.word_dim)).astype())).cuda(
                        int(args.gpu)))
            else:
                model.word_embedding.weight.data.set_(
                    torch.FloatTensor((np.zeros((utils.get_num_words() + 1, args.word_dim))).astype(float)))

    if args.train_embeddings == False:
        model.word_embedding.weight.requires_grad = False

    params = [p for p in model.parameters() if p.requires_grad]
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=params, lr=args.lr, weight_decay=args.l2_coeff)
    model.zero_grad()
    scheduler = StepLR(optimizer, step_size=50, gamma=0.9)
    print('Loading sets...')
    dev_set = utils.get_split_data(split='dev')
    train_set = utils.get_split_data(split='train')
    train_set =train_set[0:int(len(train_set)/2)]
    print('Train and dev sets loaded')

    def train():
        prev_accuracy = 0
        model.train()
        for step in range(args.step_num + 1):
            random.shuffle(train_set)
            docs = train_set[0:args.batch_size]
            labels = Variable(torch.LongTensor([d['label'] for d in docs]))
            doc_encodings, _ = model(docs)
            optimizer.zero_grad()
            outputs = doc_encodings.cpu()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()
            print("Step {} Loss {}".format(step, loss.data))
            train_performance_log.write("{},{}\n".format(step, loss.data))
            if step % 20 == 0 and step:
                accuracy = evaluation(step)
                if accuracy >= prev_accuracy:
                    torch.save(model, save_path)
                    print("Best model saved in {} Accuracy {}".format(save_path, accuracy))
                    prev_accuracy = accuracy

    def evaluation(step):
        print('Start evaluation ...')
        model.eval()
        eval_labels = [d['label'] for d in dev_set]
        labels = Variable(torch.LongTensor(eval_labels))
        doc_encodings, _ = model(dev_set)
        outputs = doc_encodings.cpu()
        loss = criterion(outputs, labels).data
        _, predictions = torch.max(outputs.data, 1)
        predictions = predictions.numpy()

        accuracy = accuracy_score(y_true=np.array(eval_labels), y_pred=predictions)
        report = classification_report(y_true=np.array(eval_labels), y_pred=predictions, target_names=['Real', 'Fake'])
        conf_matrix = confusion_matrix(y_true=np.array(eval_labels), y_pred=predictions)
        eval_performance_log.write("Step {}, Loss {} Accuracy {} \n".format(step, loss, accuracy))
        eval_performance_log.write("{}\n".format(report))
        eval_performance_log.write("{}\n".format(conf_matrix))
        eval_performance_log.write("{}\n".format('=' * 50))

        print('************* Evaluation ****************')
        print("Step {}, Loss {}  Accuracy {} ".format(step, loss, accuracy))
        print(report)
        print(conf_matrix)
        print('*****************************************')
        return accuracy

    train()


run()

No pre-trained model
Loading sets...
Train and dev sets loaded
Step 0 Loss 0.6735537455466525
Step 1 Loss 0.6836371382493723
Step 2 Loss 0.6925395727157593
Step 3 Loss 0.9074100255966187
Step 4 Loss 0.6778961620059204
Step 5 Loss 0.6685147285461426
Step 6 Loss 0.6735104322433472
Step 7 Loss 0.6663330628612129
Step 8 Loss 0.6785815954208374
Step 9 Loss 0.6897897124290466
Step 10 Loss 0.6447967290878296
Step 11 Loss 0.6577913165092468
Step 12 Loss 0.6219757199287415
Step 13 Loss 0.6282685399055481
Step 14 Loss 0.6634389162063599
Step 15 Loss 0.7681278586387634
Step 16 Loss 0.6670557360513306
Step 17 Loss 0.5521855354509082
Step 18 Loss 0.6398436427116394
Step 19 Loss 0.6271913163292469
Step 20 Loss 0.6171112461242562
Start evaluation ...
************* Evaluation ****************
Step 20, Loss 0.6367923617362976   Accuracy 0.599756488439701
              precision  recall    f1-score  support
       Real       0.64      0.40      0.50        67
       Fake       0.57      0.78      0.65  